In [2]:
import importlib
import models.mip_nerf as mipnerf
import torch

In [3]:
torch.__version__

'1.10.1+cu113'

In [4]:
# importlib.reload(mipnerf)

In [5]:
import pytorch_lightning as pl
from data import MultiscenesDataModule
from util.options import parse_custom_options
import shlex

In [6]:
args = """
--train_dataroot '/home/benjs/data_uORF/1200shape_50bg'  --test_dataroot "" \
    --n_scenes 1 --n_img_each_scene 4 --display_grad \
    --load_size 128 --n_samp 64 --input_size 128 --supervision_size 64 --coarse_epoch 120 \
    --no_locality_epoch 60 --z_dim 64 --num_slots 5 --bottom \
    --batch_size 1 --num_threads 10 \
"""

In [7]:
%%capture
opt = parse_custom_options(shlex.split(args))
pl.seed_everything(opt.seed)

Global seed set to 2021


In [8]:

dataset = MultiscenesDataModule(opt)  # create a dataset given opt.dataset_mode and other options
dataset.setup("fit")

Fetching train dataset ...


In [27]:
importlib.reload(mipnerf)
model = mipnerf.uorfMipNerf()
proj = mipnerf.MipProjection(frustum_size=[10, 10, 64])

In [29]:
model.to('cuda')

for batch_idx, batch in enumerate(dataset.train_dataloader()):
    imgs, cam2world, cam2world_azi = batch

    imgs = imgs.to('cuda')
    cam2world = cam2world.to('cuda')
    cam2world_azi = cam2world_azi.to('cuda')
    
    B, S, C, H, W = imgs.shape

    if opt.fixed_locality:
        nss2cam0 = cam2world[:, 0:1].inverse()
    else:
        print("fixed locality")
        nss2cam0 = cam2world_azi[:, 0:1].inverse()
    
    N = B*S
    cam2world = cam2world.view(N, 4, 4)
    rays = proj.get_rays(cam2world)

    z_vals = torch.ones((1, 5, 64), device=cam2world.device)

    out = model(z_vals, rays)
    

fixed locality
torch.Size([4, 1, 100, 33])
torch.Size([4, 1, 100, 64])
torch.Size([4, 4, 100, 33])
torch.Size([4, 4, 100, 64])


In [37]:
a = torch.ones(3, 4) * torch.arange(4)
a[..., 3:4]

tensor([[3.],
        [3.],
        [3.]])

In [39]:
t_vals, samples = mipnerf.sample_along_rays(
        rays.origins.flatten(end_dim=-2),
        rays.directions.flatten(end_dim=-2),
        rays.radii.flatten(end_dim=-2),
        64,
        rays.near.flatten(end_dim=-2),
        rays.far.flatten(end_dim=-2)
    )

samples_enc = mipnerf.integrated_pos_enc(
    samples,
    0,
    5
)

viewdirs_enc = mipnerf.pos_enc(
    rays.viewdirs,
    0,
    5
)  # [n_scenes*n_rays, 2*3*max_exp + 3]

print(samples_enc.shape)
print(viewdirs_enc.shape)
print(samples[0].shape)
print(samples[1].shape)
print(t_vals.shape)


torch.Size([400, 64, 30])
torch.Size([4, 10, 10, 33])
torch.Size([400, 64, 3])
torch.Size([400, 64, 3])
torch.Size([400, 65])


In [19]:
print(samples[1][0, 0])
print(samples[1][0, 32])
print(samples[1][0, 63])

tensor([0.0041, 0.0010, 0.0009], device='cuda:0')
tensor([0.0044, 0.0021, 0.0021], device='cuda:0')
tensor([0.0049, 0.0040, 0.0040], device='cuda:0')


In [21]:
print(samples[1][8000, 0])
print(samples[1][8000, 32])
print(samples[1][8000, 63])

tensor([0.0022, 0.0004, 0.0033], device='cuda:0')
tensor([0.0030, 0.0017, 0.0038], device='cuda:0')
tensor([0.0043, 0.0038, 0.0046], device='cuda:0')


In [100]:
print(model)

MipNerf(
  (mlp): MipMLP(
    (before_skip): Sequential(
      (0): Linear(in_features=94, out_features=64, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU(inplace=True)
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): ReLU(inplace=True)
    )
    (after_skip): Sequential(
      (0): Linear(in_features=158, out_features=64, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU(inplace=True)
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): ReLU(inplace=True)
    )
    (to_density): Linear(in_features=64, out_features=1, bias=True)
    (condition_bottleneck): Linear(in_features=64, out_features=64, bias=True)
    (condition_align): Sequential(
      (0): Linear(in_features=97, out_features=32, bias=True)
      (1): ReLU(inplace=True)
    )
    (to_rgb): Linear(in_features=32, out_features=3, bias=True)
  )
